<a href="https://colab.research.google.com/github/josejustinos/projeto_alura/blob/main/projeto_alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PdfReader

In [ ]:
import os
import requests
from PyPDF2 import PdfReader

# Bloco de funções auxiliares
def carregar_arquivo():
    # Solicita ao usuário para inserir o caminho do arquivo PDF
    caminho_arquivo = input('Por favor, insira o caminho do arquivo PDF: ')
    return caminho_arquivo

def analisar_conteudo(conteudo, gemini_api_key):
    # Configurar o endpoint do Google Gemini e a chave da API
    endpoint = "https://vision.googleapis.com/v1/files:asyncBatchAnnotate"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {gemini_api_key}"
    }

    # Corpo da requisição com o conteúdo do PDF em base64
    payload = {
        "requests": [
            {
                "inputConfig": {
                    "content": conteudo,
                    "mime_type": "application/pdf"
                },
                "features": [{"type": "DOCUMENT_TEXT_DETECTION"}]
            }
        ]
    }

    # Fazer a requisição ao Gemini para análise do arquivo
    response = requests.post(endpoint, headers=headers, json=payload)
    response.raise_for_status()  # Levanta um erro se a requisição falhou
    return response.json()

def buscar_informacoes(texto):
    # Buscar pelas informações desejadas de concurso público no texto extraído
    informacoes = {}

    if "inscrição para concurso público" in texto or "vagas para concurso público" in texto:
        informacoes['vagas'] = buscar_trecho(texto, "Vagas disponíveis:")
        informacoes['escolaridade'] = buscar_trecho(texto, "Escolaridade exigida:")
        informacoes['data_inicial'] = buscar_trecho(texto, "Data inicial para inscrição:")
        informacoes['data_final'] = buscar_trecho(texto, "Data final para inscrição:")
        informacoes['valor_inscricao'] = buscar_trecho(texto, "Valor da inscrição:")
        informacoes['local_inscricao'] = buscar_trecho(texto, "Local para inscrição:")
    return informacoes

def buscar_trecho(texto, chave):
    # Função para buscar um trecho no texto com base na chave fornecida
    inicio = texto.find(chave)
    if inicio == -1:  # Se a chave não for encontrada, retornar que não foi encontrada
        return f"{chave} não encontrada."
    else:
        inicio += len(chave)
        fim = texto.find('\n', inicio)  # Assumindo que cada informação está em uma linha separada
        return texto[inicio:fim].strip() if fim != -1 else texto[inicio:].strip()

def imprimir_informacoes(informacoes):
    # Imprime as informações encontradas, uma por linha
    for chave, valor in informacoes.items():
        print(f"{chave}: {valor}")

def loop_principal():
    gemini_api_key = 'SUA_API_KEY'

    while True:
        # Solicitar ao usuário para carregar o arquivo PDF
        arquivo = carregar_arquivo()

        # Verificar se o arquivo existe
        if not os.path.exists(arquivo):
            print('Arquivo não encontrado. Tente novamente.')
            continue

        # Lendo o conteúdo do PDF
        with open(arquivo, "rb") as file:
            reader = PdfReader(file)
            num_pages = len(reader.pages)
            texto = ''
            for page in reader.pages:
                texto += page.extract_text() + "\n"

        # Faz a chamada para o Google Gemini enviando o conteúdo do PDF
        resultado_analise = analisar_conteudo(texto, gemini_api_key)

        # Busca as informações necessárias no texto retornado pelo Google Gemini
        informacoes = buscar_informacoes(texto)

        # Se informações foram encontradas, imprimir, senão informar ao usuário
        if informacoes:
            imprimir_informacoes(informacoes)
        else:
            print("Não foram encontradas vagas de concurso público no arquivo PDF.")

        # Perguntar ao usuário se quer carregar outro arquivo ou encerrar o programa
        escolha = input('Deseja carregar outro arquivo? (s/n): ')
        if escolha.lower() != 's':
            break

# Executando o loop principal
if __name__ == "__main__":
    loop_principal()